## Data input for BIDS datasets
`DataGrabber` and `SelectFiles` are great if you are dealing with generic datasets with arbitrary organization. However if you have decided to use Brain Imaging Data Structure (BIDS) to organized your data (or got your hands on a BIDS dataset) you can take advanted of a formal structure BIDS imposes. In this short tutorial you will learn how to do this.

## `pybids` - a Python API for working with BIDS datasets
`pybids` is a lightweight python API for querying BIDS folder structure for specific files and metadata. You can install it from PyPi:
```
pip install pybids
```
Please note it should be already installed in the tutorial Docker image.

## The `layout` object and simple queries
To begin working with pubids we need to initalize a layout object. We will need it to do all of our queries

In [ ]:
from bids.grabbids import BIDSLayout
layout = BIDSLayout("/data/ds000114/")

In [ ]:
#!tree /data/ds000114/

Let's figure out what are the subject labels in this dataset

In [ ]:
layout.get_subjects()

['01', '02']

What modalities are included in this dataset?

In [ ]:
layout.get_modalities()

['anat', 'dwi', 'func']

What different data types are included in this dataset?

In [ ]:
layout.get_types()

['bet', 'bold', 'brain', 'dwi', 'events', 'mask', 'smooth', 'T1w']

In [ ]:
layout.get_types(modality='func')

['bold', 'events']

What are the different tasks included in this dataset?

In [ ]:
layout.get_tasks()

['fingerfootlips', 'linebisection']

We can also ask for all of the data for a particular subject and one modality.

In [ ]:
layout.get(subject='01', modality="anat")

[File(filename='/data/ds000114/sub-01/anat/sub-01_T1w.nii.gz', type='T1w', subject='01', modality='anat'),
 File(filename='/data/ds000114/sub-01/anat/sub-01_T1w_bet.nii.gz', type='bet', subject='01', modality='anat'),
 File(filename='/data/ds000114/sub-01/anat/sub-01_T1w_bet_mask.nii.gz', type='mask', subject='01', modality='anat'),
 File(filename='/data/ds000114/sub-01/anat/T1w_nipype_bet.nii.gz', subject='01', modality='anat'),
 File(filename='/data/ds000114/sub-01/anat/T1w_nipype_bet_mask.nii.gz', subject='01', modality='anat')]

We can also ask for a specific subset of data. Note that we are using extension filter to get just the imaging data (BIDS allows both .nii and .nii.gz so we need to include both).

In [ ]:
layout.get(subject='01', type='bold', extensions=['nii', 'nii.gz'])

[File(filename='/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz', type='bold', task='fingerfootlips', subject='01', modality='func'),
 File(filename='/data/ds000114/sub-01/func/sub-01_task-linebisection_bold.nii.gz', type='bold', task='linebisection', subject='01', modality='func')]

You probably noticed that this method does not only return the file paths, but objects with relevant query fields. We can easily extract just the file paths.

In [ ]:
[f.filename for f in layout.get(subject='01', type='bold', extensions=['nii', 'nii.gz'])]

['/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz',
 '/data/ds000114/sub-01/func/sub-01_task-linebisection_bold.nii.gz']

### Exercise 1:
List all files for the "linebisection" task for subject 02.

## Including `pybids` in your `nipype` workflow
This is great, but what we really want is to include this into our `nipype` workflows. How to do this? We can create our own custom `BIDSDataGrabber` using a `Function` Interface. First we need a plain Python function that for a given subject label and dataset location will return list of BOLD files.

In [ ]:
def get_niftis(subject_id, data_dir):
    # Remember that all the necesary imports need to be INSIDE the function for the Function Interface to work!
    from bids.grabbids import BIDSLayout
    
    layout = BIDSLayout(data_dir)
    
    bolds = [f.filename for f in layout.get(subject=subject_id, type="bold", extensions=['nii', 'nii.gz'])]
    
    return bolds

In [ ]:
get_niftis('01', '/data/ds000114')

['/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz',
 '/data/ds000114/sub-01/func/sub-01_task-linebisection_bold.nii.gz']

Ok we got our function. Now we need to wrap it inside a Node object.

In [ ]:
from nipype.pipeline import Node, MapNode, Workflow
from nipype.interfaces.utility import IdentityInterface, Function

In [ ]:
BIDSDataGrabber = Node(Function(function=get_niftis, input_names=["subject_id",
                                       "data_dir"],
                                   output_names=["bolds"]), name="BIDSDataGrabber")
BIDSDataGrabber.inputs.data_dir = "/data/ds000114"

In [ ]:
BIDSDataGrabber.inputs.subject_id='01'
res = BIDSDataGrabber.run()
res.outputs

170715-20:06:48,67 workflow INFO:
	 Executing node BIDSDataGrabber in dir: /tmp/tmpxmaghas0/BIDSDataGrabber



bolds = ['/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz', '/data/ds000114/sub-01/func/sub-01_task-linebisection_bold.nii.gz']

Works like a charm! (hopefully :) Lets put it in a workflow. We are not going to analyze any data, but for demostrantion purposes we will add a couple of nodes that pretend to analyze their inputs

In [ ]:
def printMe(paths):
    print("\n\nanalyzing " + str(paths) + "\n\n")
    
analyzeBOLD = Node(Function(function=printMe, input_names=["paths"],
                            output_names=[]), name="analyzeBOLD")

In [ ]:
wf = Workflow(name="bids_demo")
wf.connect(BIDSDataGrabber, "bolds", analyzeBOLD, "paths")
wf.run()

170715-20:06:48,655 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170715-20:06:48,689 workflow INFO:
	 Running serially.
170715-20:06:48,691 workflow INFO:
	 Executing node BIDSDataGrabber in dir: /tmp/tmp3oj49wl6/bids_demo/BIDSDataGrabber
170715-20:06:49,161 workflow INFO:
	 Executing node analyzeBOLD in dir: /tmp/tmpg1p720ko/bids_demo/analyzeBOLD


analyzing ['/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz', '/data/ds000114/sub-01/func/sub-01_task-linebisection_bold.nii.gz']




### Exercise 2:
Modify the `BIDSDataGrabber` and the workflow to include T1ws.

## Iterating over subject labels
In the previous example we demostrated how to use `pybids` to "analyze" one subject. How can we scale it for all subjects? Easy - using `iterables` (more in [Iteration/Iterables](basic_iteration.ipynb).

In [ ]:
BIDSDataGrabber.iterables = ('subject_id', layout.get_subjects())
wf.run()

170715-20:06:49,195 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170715-20:06:49,232 workflow INFO:
	 Running serially.
170715-20:06:49,234 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a1 in dir: /tmp/tmp04hcd8b3/bids_demo/_subject_id_02/BIDSDataGrabber
170715-20:06:49,661 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a0 in dir: /tmp/tmpjx_5r1xs/bids_demo/_subject_id_01/BIDSDataGrabber
170715-20:06:50,88 workflow INFO:
	 Executing node analyzeBOLD.a0 in dir: /tmp/tmplisz8wgv/bids_demo/_subject_id_01/analyzeBOLD


analyzing ['/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz', '/data/ds000114/sub-01/func/sub-01_task-linebisection_bold.nii.gz']


170715-20:06:50,109 workflow INFO:
	 Executing node analyzeBOLD.a1 in dir: /tmp/tmpypfcf6g3/bids_demo/_subject_id_02/analyzeBOLD


analyzing ['/data/ds000114/sub-02/func/sub-02_task-fingerfootlips_bold.nii.gz', '/data/ds000114/sub-02/func/sub-02_task-linebisection_bold.nii.gz']




## Accessing additional metadata
Querying different files is nice, but sometimes you want to access more metadata. For example `RepetitionTime`. `pybids` can help with that as well

In [ ]:
layout.get_metadata('/data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz')

{'EchoTime': 0.05,
 'FlipAngle': 90,
 'RepetitionTime': 2.5,
 'SliceTiming': [0.0,
  1.2499999999999998,
  0.08333333333333333,
  1.333333333333333,
  0.16666666666666666,
  1.4166666666666663,
  0.25,
  1.4999999999999996,
  0.3333333333333333,
  1.5833333333333328,
  0.41666666666666663,
  1.666666666666666,
  0.5,
  1.7499999999999993,
  0.5833333333333333,
  1.8333333333333326,
  0.6666666666666666,
  1.9166666666666659,
  0.75,
  1.9999999999999991,
  0.8333333333333333,
  2.083333333333332,
  0.9166666666666666,
  2.1666666666666656,
  1.0,
  2.249999999999999,
  1.0833333333333333,
  2.333333333333332,
  1.1666666666666665,
  2.416666666666665],
 'TaskName': 'finger_foot_lips'}

Can we incorporate this into our pipeline? Yes we can!
More about MapNode in [MapNode](basic_mapnodes.ipynb)

In [ ]:
def printMetadata(path, data_dir):
    from bids.grabbids import BIDSLayout
    layout = BIDSLayout(data_dir)
    print("\n\nanalyzing " + path + "\nTR: "+ str(layout.get_metadata(path)["RepetitionTime"]) + "\n\n")
    
analyzeBOLD2 = MapNode(Function(function=printMetadata, input_names=["path", "data_dir"],
                             output_names=[]), name="analyzeBOLD2", iterfield="path")
analyzeBOLD2.inputs.data_dir = "/data/ds000114/"

In [ ]:
wf = Workflow(name="bids_demo")
wf.connect(BIDSDataGrabber, "bolds", analyzeBOLD2, "path")
wf.run()

170715-20:06:50,226 workflow INFO:
	 Workflow bids_demo settings: ['check', 'execution', 'logging']
170715-20:06:50,261 workflow INFO:
	 Running serially.
170715-20:06:50,264 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a0 in dir: /tmp/tmpuurnyq3s/bids_demo/_subject_id_01/BIDSDataGrabber
170715-20:06:50,731 workflow INFO:
	 Executing node analyzeBOLD2.a0 in dir: /tmp/tmp2dwr9m2c/bids_demo/_subject_id_01/analyzeBOLD2
170715-20:06:50,750 workflow INFO:
	 Executing node _analyzeBOLD20 in dir: /tmp/tmp2dwr9m2c/bids_demo/_subject_id_01/analyzeBOLD2/mapflow/_analyzeBOLD20


analyzing /data/ds000114/sub-01/func/sub-01_task-fingerfootlips_bold.nii.gz
TR: 2.5


170715-20:06:51,229 workflow INFO:
	 Executing node _analyzeBOLD21 in dir: /tmp/tmp2dwr9m2c/bids_demo/_subject_id_01/analyzeBOLD2/mapflow/_analyzeBOLD21


analyzing /data/ds000114/sub-01/func/sub-01_task-linebisection_bold.nii.gz
TR: 2.5


170715-20:06:51,733 workflow INFO:
	 Executing node BIDSDataGrabber.aI.a1 in dir: /tmp/tmpj4g

### Exercise 3:
Modify the `printMetadata` function to also print `EchoTime` 